# Google Landmark Recognition Challenge 2020
Simplified image similarity ranking and re-ranking implementation with:
* EfficientNetB0 backbone for global feature similarity search
* DELF module for local feature reranking

Reference papers:
* 2020 Recognition challenge winner: https://arxiv.org/abs/2010.01650
* 2019 Recognition challend 2nd place: https://arxiv.org/abs/1906.03990

In [ ]:
!nvidia-smi

In [ ]:
# Importing libraries
import os
import cv2
import shutil
import numpy as np
import pandas as pd
from scipy import spatial
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
# Directories and file paths
TRAIN_DIR = '../input/landmark-recognition-2020/train'
TRAIN_CSV = '../input/landmark-recognition-2020/train.csv'
train_df = pd.read_csv(TRAIN_CSV)

TRAIN_PATHS = [os.path.join(TRAIN_DIR, f'{img[0]}/{img[1]}/{img[2]}/{img}.jpg') for img in train_df['id']]
train_df['path'] = TRAIN_PATHS

train_df

In [ ]:
# Subsetting
train_df_grouped = pd.DataFrame(train_df.landmark_id.value_counts())
train_df_grouped.reset_index(inplace=True)
train_df_grouped.columns = ['landmark_id','count']

# Selected landmarks based on inclass frequency
selected_landmarks = train_df_grouped[(train_df_grouped['count'] <= 155) & (train_df_grouped['count'] >= 150)]

train_df_sub = train_df[train_df['landmark_id'].isin(selected_landmarks['landmark_id'])]
new_id = []
current_id = 0
previous_id = int(train_df_sub.head(1)['landmark_id'])
for landmark_id in train_df_sub['landmark_id']:
    if landmark_id == previous_id:
        new_id.append(current_id)
    else:
        current_id += 1
        new_id.append(current_id)
        previous_id = landmark_id

train_df_sub['new_id'] = new_id

NUM_CLASSES = train_df_sub['landmark_id'].nunique()

print(f"Unique classes found: {NUM_CLASSES}")
train_df_sub

In [ ]:
# Training and validation splits
# 90/10 stratified split for training and validation
X_train, X_val, y_train, y_val = train_test_split(train_df_sub[['id', 'path']], train_df_sub['new_id'],
                                                  train_size = 0.9,
                                                  random_state = 123,
                                                  shuffle = True,
                                                  stratify = train_df_sub['new_id'])

# Held-out test set for inference
# Further 95/5 split -> 5% of original training set left for test set
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                                   train_size = 0.95,
                                                   random_state = 123,
                                                   shuffle = True,
                                                   stratify = y_train)

assert X_train.shape[0] + X_val.shape[0] + X_test.shape[0] == train_df_sub.shape[0]

print(f"Training data shape: {X_train.shape}")
print(f"Training label shape: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Validation label shape: {y_val.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test label shape: {y_test.shape}")

In [ ]:
print(f"Unique classes on y_train: {y_train.nunique()}")
print(f"Unique classes on y_val: {y_val.nunique()}")
print(f"Unique classes on y_test: {y_test.nunique()}")

In [ ]:
# Classes distribution on training, validation and test sets
plt.figure(figsize = (10, 3))
ax = sns.histplot(y_train, bins=75, kde = True)
ax.set_title('Distribution of Landmarks on training set')
plt.tight_layout()

plt.figure(figsize = (10, 3))
ax = sns.histplot(y_val, bins=75, kde = True)
ax.set_title('Distribution of Landmarks on validation set')
plt.tight_layout()

plt.figure(figsize = (10, 3))
ax = sns.histplot(y_test, bins=75, kde = True)
ax.set_title('Distribution of Landmarks on test set')
plt.tight_layout()
plt.show()

In [ ]:
!rm -r train_sub, test_sub, val_sub && ls

In [ ]:
# Creating image directories for classes subset
NEW_BASE_DIR = "/kaggle/working"

# Training set directory
for file, path, landmark in tqdm(zip(X_train['id'], X_train['path'], y_train)):
    dir = f"{NEW_BASE_DIR}/train_sub/{str(landmark)}"
    os.makedirs(dir, exist_ok = True)
    fname = f"{file}.jpg"
    shutil.copyfile(src = path, dst = f"{dir}/{fname}")

# Validation set directory    
for file, path, landmark in tqdm(zip(X_val['id'], X_val['path'], y_val)):
    dir = f"{NEW_BASE_DIR}/val_sub/{str(landmark)}"
    os.makedirs(dir, exist_ok = True)
    fname = f"{file}.jpg"
    shutil.copyfile(src = path, dst = f"{dir}/{fname}")

# Training set directory
for file, path, landmark in tqdm(zip(X_test['id'], X_test['path'], y_test)):
    dir = f"{NEW_BASE_DIR}/test_sub/{str(landmark)}"
    os.makedirs(dir, exist_ok = True)
    fname = f"{file}.jpg"
    shutil.copyfile(src = path, dst = f"{dir}/{fname}")

In [ ]:
!ls

In [ ]:
!cd train_sub && ls

In [ ]:
# Creating tensorflow tf.data.Dataset
from tensorflow.keras.utils import image_dataset_from_directory

IMG_SIZE = 224
BATCH_SIZE = 16

print("Building training dataset...")
# Training tf.data.Dataset
train_ds = image_dataset_from_directory(f"{NEW_BASE_DIR}/train_sub",
                                        label_mode = 'int',
                                        shuffle = True,
                                        image_size = (IMG_SIZE, IMG_SIZE),
                                        batch_size = BATCH_SIZE)

print("Building validation dataset...")
# Validation tf.data.Dataset
val_ds = image_dataset_from_directory(f"{NEW_BASE_DIR}/val_sub",
                                        label_mode = 'int',
                                        shuffle = True,
                                        image_size = (IMG_SIZE, IMG_SIZE),
                                        batch_size = BATCH_SIZE)

print("Building test dataset...")
# Test tf.data.Dataset
test_ds = image_dataset_from_directory(f"{NEW_BASE_DIR}/test_sub",
                                        label_mode = 'int',
                                        shuffle = True,
                                        image_size = (IMG_SIZE, IMG_SIZE),
                                        batch_size = BATCH_SIZE)

In [ ]:
# Visualizing a random batch from training dataset
for data_batch, labels_batch in train_ds.take(1):
    ncols = 4
    nrows = int(data_batch.shape[0]/ncols)
    fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize=(10, 11),
                           sharex = True, sharey = True)
    img_counter = 0
    for image, label in zip(data_batch, labels_batch):
        axi = ax.flat[img_counter]
        axi.imshow(image/255.)
        label = label.numpy()
#         axi.set_title(np.where(label == 1)[0])
        axi.set_title(label)
        img_counter += 1
plt.show()

In [ ]:
####### ALTERNATIVE CODE FOR UNBATCHED DATASET #######
# ncols = 4
# nrows = 4
# fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize=(10, 11),
#                        sharex = True, sharey = True)
# img_counter = 0
# for image, label in train_ds.take(16):
#     axi = ax.flat[img_counter]
#     axi.imshow(image[0]/255.)
#     label = label.numpy()
# #         axi.set_title(np.where(label == 1)[0])
#     axi.set_title(label)
#     img_counter += 1
# plt.show()

In [ ]:
# Defining a data augmentation stage
img_augmentation = tf.keras.Sequential(
    # [layers.RandomFlip("horizontal"),
    [layers.RandomTranslation(height_factor = 0.1, width_factor = 0.1),
     layers.RandomRotation(0.02),
     layers.RandomZoom(0.2)],
     name = "img_augmentation",
)

In [ ]:
# Displaying variations of a randomly augmented training image
plt.figure(figsize=(9, 9))
for image, label in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = img_augmentation(image, training = True)
        plt.imshow(augmented_image[15].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
####### ALTERNATIVE CODE FOR UNBATCHED DATASET #######
# # Displaying variations of a randomly augmented training image
# plt.figure(figsize=(9, 9))
# for image, label in train_ds.take(16):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         augmented_image = img_augmentation(image[0], training = True)
#         plt.imshow(augmented_image.numpy().astype("uint8"))
#         plt.axis("off")

In [ ]:
# Model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

MODELS_DIR = f"{NEW_BASE_DIR}/models"

os.makedirs(MODELS_DIR, exist_ok = True)

# Model instantiator
def build_model(num_classes = None):
    inputs = keras.Input(shape = (IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    # EfficientNetB0 backbone
    model = EfficientNetB0(input_tensor = x,
                           weights = 'imagenet',
                           include_top = False,
                           drop_connect_rate = DROP_CONNECT_RATE)
    
    # Freeze pretrained weights
    model.trainable = False
    
    # Rebuild top
    x = layers.GlobalAveragePooling2D(name = "avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(TOP_DROPOUT_RATE, name = "top_dropout")(x)
    
    # Embedding
    embedding = layers.Dense(512, name = "embedding_512")(x)
    outputs = layers.Dense(num_classes, activation = "softmax", name = "softmax")(embedding)
    
    # Compile
    model = tf.keras.Model(inputs, outputs, name = "EfficientNetB0")
    optimizer = tf.keras.optimizers.Adam(learning_rate = ADAM_LR)
    model.compile(optimizer = optimizer,
                 loss = "sparse_categorical_crossentropy",
                 metrics = ["accuracy"])
    
    return model

In [ ]:
# # Model summaries
# tf.keras.utils.plot_model(model)
# model.summary()

In [ ]:
# Training history visualization
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
# Instantiating model
# Hyperparameters
DROP_CONNECT_RATE = 0.2 # Dropout rate for stochastic depth on EfficientNet
TOP_DROPOUT_RATE = 0.2  # Top dropout
INIT_LR = 5e-3          # Initial learning rate
EPOCHS = 20
# Adam optimizer learning rate schedule
ADAM_LR = tf.keras.optimizers.schedules.ExponentialDecay(
    INIT_LR,
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

model = build_model(num_classes = NUM_CLASSES)

In [ ]:
!ls

In [ ]:
# Training embedding layer
model_file_path = os.path.join(MODELS_DIR, "EfficientNetB0_softmax.keras")
callbacks = [
    keras.callbacks.ModelCheckpoint(model_file_path,
                                    save_best_only=True,
                                    monitor = "val_accuracy"),
    keras.callbacks.EarlyStopping(patience = 2,
                                  monitor = "val_accuracy")]

hist = model.fit(train_ds,
                 epochs = EPOCHS,
                 validation_data = val_ds,
                 shuffle = 'batch',
                 callbacks = callbacks)

plot_hist(hist)

In [ ]:
# Evaluating best model
model = keras.models.load_model(model_file_path)
print("Predictions on validation set...")
print(f"Validation accuracy: {model.evaluate(val_ds)[1]*100:.2f} %")
print("Predictions on test set...")
print(f"Test accuracy: {model.evaluate(test_ds)[1]*100:.2f} %")

### EfficientNetB7

In [ ]:
# Evaluating best model
# model = keras.models.load_model(os.path.join(MODELS_DIR, "EfficientNetB7_softmax.keras"))
# print("Predictions on validation set...")
# print(f"Validation accuracy: {model.evaluate(val_ds)[1]*100:.2f} %")
# print("Predictions on test set...")
# print(f"Test accuracy: {model.evaluate(test_ds)[1]*100:.2f} %")

### EfficientNetB3

In [ ]:
# Evaluating best model
# model = keras.models.load_model(os.path.join(MODELS_DIR, "EfficientNetB3_softmax.keras"))
# print("Predictions on validation set...")
# print(f"Validation accuracy: {model.evaluate(val_ds)[1]*100:.2f} %")
# print("Predictions on test set...")
# print(f"Test accuracy: {model.evaluate(test_ds)[1]*100:.2f} %")

### EfficientNetB0

In [ ]:
# Evaluating best model
# model = keras.models.load_model(os.path.join(MODELS_DIR, "EfficientNetB0_softmax.keras"))
# print("Predictions on validation set...")
# print(f"Validation accuracy: {model.evaluate(val_ds)[1]*100:.2f} %")
# print("Predictions on test set...")
# print(f"Test accuracy: {model.evaluate(test_ds)[1]*100:.2f} %")

## Cosine Similarity
Pairwise query: key search for similarity candidates. In the following example:
* Query images: validation set
* Key images: training set

In [ ]:
# Auxiliar functions
# Load image
def get_image(path, resize = False, reshape = False, target_size = None):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if resize:
        img = cv2.resize(img, dsize = (target_size, target_size))
    if reshape:
        img = tf.reshape(img, [1, target_size, target_size, 3])
    return img

# Get landmark samples
def get_landmark(landmark_id, samples = 16):
    nrows = samples // 4
    random_imgs = np.random.choice(train_df_sub[train_df_sub['new_id'] == landmark_id].index, samples, replace = False)
    plt.figure(figsize = (12, 10))
    for i, img in enumerate(train_df_sub.loc[random_imgs, :].values):
        ax = plt.subplot(nrows, 4, i + 1)
        plt.imshow(get_image(img[2]))
        plt.title(f"{img[0]}")
        plt.suptitle(f"Samples of landmark {landmark_id}", fontsize = 14, y = 0.94, weight = "bold")
        plt.axis("off")

# Get image embeddings
def get_embeddings(model, image_paths, input_size, as_df = True):
    embeddings = {}
    embeddings['images_paths'] = []
    embeddings['embedded_images'] = []
    
    target_dir = os.path.split(os.path.split(image_paths[0])[0])[0]
    
    print(f"Retrieving embeddings for {target_dir} with {model.name}...")
    for image_path in tqdm(image_paths):
        embeddings['images_paths'].append(image_path)
        embedded_image = model.predict(get_image(image_path,
                                                 resize = True,
                                                 reshape = True,
                                                 target_size = input_size))
        embeddings['embedded_images'].append(embedded_image)
    
    if as_df:
        embeddings = pd.DataFrame(embeddings)
    
    return embeddings

# Get similarities between query key pair
def get_similarities(query, key):
    '''
    Get cosine similarity matrix between query and key pairs
    Arguments:
    query, key: embedded images
    '''
    query_array = np.stack(query.tolist()).reshape(query.shape[0],
                                                   query[0].shape[1])
    key_array = np.stack(key.tolist()).reshape(key.shape[0],
                                               key[0].shape[1])
    
    # Initializing similarity matrix
    similarity = np.zeros((query_array.shape[0], key_array.shape[0]))
    
    # Getting pairwise similarities
    print(f"Getting pairwise {query_array.shape[0]} query: {key_array.shape[0]} key similarities...")
    for query_index in tqdm(range(query_array.shape[0])):
        similarity[query_index] = 1 - spatial.distance.cdist(query_array[np.newaxis, query_index, :],
                                                             key_array,
                                                             'cosine')[0]
    return similarity

# Plot top ranked images
def plot_similar(similar_imgs, img_paths):
    '''
    Plot top N similar samples from similarity index
    '''
    plt.figure(figsize = (18, 6))
    nrows = similar_imgs.shape[0]//5
    for i, img in enumerate(similar_imgs):
        ax = plt.subplot(nrows, 5, i + 1)
        plt.imshow(get_image(img_paths[img]))
        plt.title(f"Landmark id: {os.path.split(os.path.split(img_paths[img])[0])[1]}")
        plt.axis("off")

In [ ]:
# Embedding models
embedding_layer = 'embedding_512'
embedding_model = tf.keras.Model(inputs = model.input,
                                 outputs = model.get_layer(embedding_layer).output,
                                 name = "EfficientNetB0_embed512")

In [ ]:
# Retrieving embeddings
train_img_paths = train_ds.file_paths
val_img_paths = val_ds.file_paths

train_embeddings = get_embeddings(model = embedding_model,
                                 image_paths = train_img_paths,
                                 input_size = IMG_SIZE)

val_embeddings = get_embeddings(model = embedding_model,
                                 image_paths = val_img_paths,
                                 input_size = IMG_SIZE)

In [ ]:
train_embeddings.head()

In [ ]:
val_embeddings.head()

In [ ]:
val_train_similarity = get_similarities(val_embeddings['embedded_images'],
                                        train_embeddings['embedded_images'])
val_train_similarity.shape

In [ ]:
# Calculating confidence score per submission
def confidence_top(query = None, key = None, similarity = None, query_image_index = None, top = 5):
    '''
    Arguments:
    query_image_index = index of query image on similarity matrix query axis
    Return confidence scores for top N predictions
    '''
    query_paths = query['images_paths']
    key_paths = key['images_paths']
    
    similar_n = np.argsort(similarity[query_image_index])[::-1][:top]
    
    confidence_df = {}    
    confidence_df['top_similar'] = []
    for similar in similar_n:
        confidence_df['top_similar'].append(similar)

    confidence_df['image_paths'] = []
    for similar in similar_n:
        similar_image_path = key_paths[similar]
        confidence_df['image_paths'].append(similar_image_path)    
        
    confidence_df['prediction'] = []
    for similar in similar_n:
        similar_image_path = key_paths[similar]
        y = int(os.path.split(os.path.split(similar_image_path)[0])[1])
        confidence_df['prediction'].append(y)  
    
    confidence_df['cos_similarity'] = []
    for similar in similar_n:
        confidence_df['cos_similarity'].append(similarity[query_image_index][similar]) 
    
    return pd.DataFrame(confidence_df)

In [ ]:
query_image_index = 0
top_n = 5

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

In [ ]:
query_image_index = 4
top_n = 5

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

In [ ]:
query_image_index = 5
top_n = 5

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

In [ ]:
query_image_index = 11
top_n = 5

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

In [ ]:
query_image_index = 92
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

## Object oclusion example
Object oclusion is only one of the examples of how a local feature reranking method improves query performance

In [ ]:
query_image_index = 573
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

## DELF module
Local features search

References:
* Large-Scale Image Retrieval with Attentive Deep Local Features: https://arxiv.org/abs/1612.06321
* DELF on Tensorflow Hub: https://github.com/tensorflow/models/tree/master/research/delf


In [ ]:
DELF_IMG_SIZE = 600

In [ ]:
image_1 = get_image(val_embeddings['images_paths'][573],
                    resize = True,
                    target_size = DELF_IMG_SIZE)

plt.figure(figsize = (6, 6))
plt.imshow(image_1)
plt.axis("off")
plt.show()

In [ ]:
image_2 = get_image(train_embeddings['images_paths'][similar_n[5]],
                     resize = True,
                     target_size = DELF_IMG_SIZE)

plt.figure(figsize = (6, 6))
plt.imshow(image_2)
plt.axis("off")
plt.show()

In [ ]:
from absl import logging
from PIL import Image, ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
from six import BytesIO

import tensorflow_hub as hub
from six.moves.urllib.request import urlopen

In [ ]:
delf = hub.load('https://tfhub.dev/google/delf/1').signatures['default']

In [ ]:
# DELF module
def run_delf(image):
    '''
    Apply DELF module to the input image
    Arguments:
    image: np.array resized image
    '''
    float_image = tf.image.convert_image_dtype(image, tf.float32)

    return delf(
      image = float_image,
      score_threshold = tf.constant(100.0),
      image_scales = tf.constant([0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0]),
      max_feature_num = tf.constant(1000))

def match_images(image1, image2, result1, result2, verbose = True):
    distance_threshold = 0.8

    # Read features.
    num_features_1 = result1['locations'].shape[0]
    num_features_2 = result2['locations'].shape[0]
    
    if verbose:
        print("Loaded image 1's %d features" % num_features_1)
        print("Loaded image 2's %d features" % num_features_2)

    # Find nearest-neighbor matches using a KD tree.
    d1_tree = cKDTree(result1['descriptors'])
    _, indices = d1_tree.query(
      result2['descriptors'],
      distance_upper_bound=distance_threshold)

    # Select feature locations for putative matches.
    locations_2_to_use = np.array([
      result2['locations'][i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])
    locations_1_to_use = np.array([
      result1['locations'][indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])

    # Perform geometric verification using RANSAC.
    _, inliers = ransac(
      (locations_1_to_use, locations_2_to_use),
      AffineTransform,
      min_samples=3,
      residual_threshold=20,
      max_trials=1000)
    
    if verbose:
        print('Found %d inliers' % sum(inliers))

    # Visualize correspondences.
    _, ax = plt.subplots(figsize = (9, 9))
    inlier_idxs = np.nonzero(inliers)[0]
    plot_matches(
      ax,
      image1,
      image2,
      locations_1_to_use,
      locations_2_to_use,
      np.column_stack((inlier_idxs, inlier_idxs)),
      matches_color='b')
    ax.axis('off')
    ax.set_title(f'DELF correspondences: Found {sum(inliers)} inliers')

In [ ]:
delf_result1 = run_delf(image_1)
delf_result2 = run_delf(image_2)

In [ ]:
match_images(image_1, image_2, delf_result1, delf_result2)

In [ ]:
for image_index in similar_n[:6]:
    key_image = get_image(train_embeddings['images_paths'][image_index],
                          resize = True,
                          target_size = DELF_IMG_SIZE)
    delf_key_image_result = run_delf(key_image)
    match_images(image_1, key_image, delf_result1, delf_key_image_result, verbose = False)

## Reranking
Reranking using DELF local features descriptor

In [ ]:
def delf_rerank(query = None, key = None, query_image_index = None, confidence_df = None, re_sort = True):
    distance_threshold = 0.8
    query_paths = query['images_paths']
    key_paths = key['images_paths']
    
    query_image = get_image(query_paths[query_image_index],
                            resize = True,
                            target_size = DELF_IMG_SIZE)
    
    delf_result_query = run_delf(query_image)
    
    # Read query features
    num_features_query = delf_result_query['locations'].shape[0]
    
    inliers_list = []
    print(f"Retrieving local features for top {len(confidence_df['image_paths'])} key images...")
    for image_path in tqdm(confidence_df['image_paths']):
        key_image = get_image(image_path,
                          resize = True,
                          target_size = DELF_IMG_SIZE)
        
        delf_result_key = run_delf(key_image)
    
        # Read key features
        num_features_key = delf_result_key['locations'].shape[0]

        # Find nearest-neighbor matches using a KD tree.
        d1_tree = cKDTree(delf_result_query['descriptors'])
        _, indices = d1_tree.query(
          delf_result_key['descriptors'],
          distance_upper_bound=distance_threshold)

        # Select feature locations for putative matches.
        locations_k_to_use = np.array([
          delf_result_key['locations'][i,]
          for i in range(num_features_key)
          if indices[i] != num_features_query
        ])
        locations_q_to_use = np.array([
          delf_result_query['locations'][indices[i],]
          for i in range(num_features_key)
          if indices[i] != num_features_query
        ])

        # Perform geometric verification using RANSAC.
        try:
            _, inliers = ransac(
              (locations_q_to_use, locations_k_to_use),
              AffineTransform,
              min_samples=3,
              residual_threshold=20,
              max_trials=1000)
        except:
            inliers = [0]
          
        total_inliers = sum(inliers)
        inliers_list.append(total_inliers)
    
    confidence_df['inliers'] = inliers_list
    
    original_confidence = confidence_df['inliers']
    reranked_confidence = np.sqrt(original_confidence) * confidence_df['cos_similarity']
    confidence_df['reranked_conf'] = reranked_confidence
    
    if re_sort:
        confidence_df.sort_values('reranked_conf', ascending = False, inplace = True)
    
    return confidence_df

In [ ]:
reranked_df = delf_rerank(query = val_embeddings,
                          key = train_embeddings,
                          query_image_index = query_image_index,
                          confidence_df = confidence_df,
                          re_sort = True)
reranked_df

In [ ]:
query_image_index = 573
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = reranked_df['top_similar'][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

## Reranking examples

In [ ]:
query_image_index = 1
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

In [ ]:
reranked_df = delf_rerank(query = val_embeddings,
                          key = train_embeddings,
                          query_image_index = query_image_index,
                          confidence_df = confidence_df,
                          re_sort = True)
reranked_df

In [ ]:
query_image_index = 1
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = reranked_df['top_similar'][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
query_image_index = 2
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

In [ ]:
reranked_df = delf_rerank(query = val_embeddings,
                          key = train_embeddings,
                          query_image_index = query_image_index,
                          confidence_df = confidence_df,
                          re_sort = True)
reranked_df

In [ ]:
query_image_index = 2
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = reranked_df['top_similar'][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
query_image_index = 101
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

In [ ]:
confidence_df = confidence_top(query = val_embeddings,
                               key = train_embeddings,
                               similarity = val_train_similarity,
                               query_image_index = query_image_index,
                               top = top_n)

confidence_df

In [ ]:
reranked_df = delf_rerank(query = val_embeddings,
                          key = train_embeddings,
                          query_image_index = query_image_index,
                          confidence_df = confidence_df,
                          re_sort = True)
reranked_df

In [ ]:
query_image_index = 101
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = reranked_df['top_similar'][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

## Under-represented query image
Effect of querying images not well represented on the key set

In [ ]:
query_image_index = 8
top_n = 10

image_id = os.path.split(val_embeddings['images_paths'][query_image_index])[1]
query_landmark_id = os.path.split(os.path.split(val_embeddings['images_paths'][query_image_index])[0])[1]

similar_n = np.argsort(val_train_similarity[query_image_index])[::-1][:top_n]

print(f"Queried image: {image_id}")
plt.figure(figsize = (6, 6))
plt.imshow(get_image(val_embeddings['images_paths'][query_image_index]))
plt.title(f"Landmark id: {query_landmark_id}")
plt.axis("off")
plot_similar(similar_n, train_embeddings['images_paths'])

Let's investigate what the model has seen for landmark 36 during training...

In [ ]:
get_landmark(36)

Query image has a representative issue when considering how the landmark appears on key set and how it was seen during training